<a href="https://colab.research.google.com/github/JJF1962/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-30 16:18:19--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.95MB/s    in 0.2s    

2022-05-30 16:18:20 (5.95 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...| 2015-08-31|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...|           Video|          5|    

In [5]:
from pyspark.sql.functions import to_date

In [8]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35645596|             1|
|   50851028|             1|
|   20686161|             2|
|   15191478|             2|
|    2202715|             1|
|   48333068|             1|
|   22910908|             2|
|   38463562|             1|
|   26902200|             1|
|   10546543|             1|
|   33236947|             1|
|   40127650|             1|
|   41244127|             3|
|   22968117|             1|
|   25472255|             1|
|    3160356|             1|
|   18785199|             2|
|   19840529|             1|
|    1655605|             1|
|   14676950|             3|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Create the products_table Dataframe and drop duplicates
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0788806270|Bambi (Walt Disne...|
|078881107X|Kiki's Delivery S...|
|6303559018|To Catch a Yeti [...|
|6302844061|      Silkwood [VHS]|
|B00003XAMY|Wagons Roll at Ni...|
|0783216084|          Jaws [VHS]|
|6301966376|The Compleat Beat...|
|6303315429| Yoga for Life [VHS]|
|6302969204|Miracle in Milan ...|
|6301540441|As Summers Die [VHS]|
|6304611366|Loretta Young Sho...|
|1559838450|Hans Christian An...|
|6304263198|   Latcho Drom [VHS]|
|6301017250|Dead Don't Die [VHS]|
|6301782135|John Hammond: Fro...|
|6304925158|The Education of ...|
|6305403309| Dead Husbands [VHS]|
|6300152936|       Faeries [VHS]|
|B0000897E9|  Femme Fatale [VHS]|
|B0002X4MIQ|Touched By An Ang...|
+----------+--------------------+
only showing top 20 rows



In [10]:
# Create the review_id_table DataFrame
review_id_df = df.select(["review_id", "customer_id", "product_parent", to_date("review_date")])
review_id_df.show()

+--------------+-----------+--------------+----------------------+
|     review_id|customer_id|product_parent|to_date(`review_date`)|
+--------------+-----------+--------------+----------------------+
|R1P1G5KZ05H6RD|   49033728|     748506413|            2015-08-31|
|R106N066IUN8ZV|   17857748|     478710180|            2015-08-31|
| R7WTAA1S5O7D9|   25551507|     981002815|            2015-08-31|
|R32HFMVWLYOYJK|   21025041|     333219811|            2015-08-31|
| RWT3H6HBVAL6G|   40943563|     538101194|            2015-08-31|
|R1S3T3GWUGQTW7|   17013969|     716303278|            2015-08-31|
|R3R0QYHA800REE|   47611685|     134996462|            2015-08-31|
|R1FR0EQPHPW5NM|   35680737|     498116870|            2015-08-31|
| RGORN81H45NI7|   10747909|      77519275|            2015-08-31|
|R1CNYN4ABNOJSU|     126341|     917778300|            2015-08-31|
|R2DW06821PMN40|   40676812|     187850980|            2015-08-31|
|R1CS8AMA8B0VBJ|   19863533|     901056605|            2015-08